In [15]:
'''
Use the centroid locations from one camera and the estimated extrinsics
to get rough initialization coordinates for the 3D points
'''
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

import pySBA

In [6]:
# base file paths
tmp_local_root = "C:/Users/ilow1/Documents/code/bird_pose_tracking/"
root_dir = f"{tmp_local_root}calibration_files/"

centroid_dir = f"{tmp_local_root}calibration_files/preprocessed_centroids/"

# camera array file paths
cam_file_path = root_dir
init_array_folder = 'init_cam_arrays/' # to load initial estimates

In [4]:
# load the preprocessed centroid locations and indices
img_date = input("input date of calibration video acquisition (YYMMDD): ")

file_centroids = f'{img_date}_centroids.npy'
points_2d = np.load(f'{centroid_dir}{file_centroids}')

file_camera_ind = f'{img_date}_camera_ind.npy'
camera_ind = np.load(f'{centroid_dir}{file_camera_ind}')

file_point_ind = f'{img_date}_point_ind.npy'
point_ind = np.load(f'{centroid_dir}{file_point_ind}')

input date of calibration video acquisition (YYMMDD): 240716


In [5]:
file_init_3d = f'{img_date}_init_3d.npy'

In [9]:
# initialize the camera array
cam_ids = ['red_cam', 'yellow_cam', 'green_cam', 'blue_cam']
n_cams = len(cam_ids)
init_cam_array, cam_array_fields = pySBA.getCameraArray(f'{cam_file_path}{init_array_folder}', camera_ids=cam_ids)

In [13]:
# get the rotation and translation vectors
rot_vec = init_cam_array[0, :3]
translate = init_cam_array[0, 3:6]

In [ ]:
'''
to transform from the world coords W to the camera coords C, we apply the rotation and translation:
𝑊𝑅 + 𝑡 = 𝐶

so to get back from C to W, we can do the reverse:
𝑊 = (𝐶 − 𝑡) * 𝑅^-1
'''

In [ ]:
# these functions are from pySBA
def rotate(points, rot_vecs):
    """Rotate points by given rotation vectors.
    Rodrigues' rotation formula is used.
    """
    theta = np.linalg.norm(rot_vecs, axis=1)[:, np.newaxis]
    with np.errstate(invalid='ignore'):
        v = rot_vecs / theta
        v = np.nan_to_num(v)
    dot = np.sum(points * v, axis=1)[:, np.newaxis]
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)

    return cos_theta * points + sin_theta * np.cross(v, points) + dot * (1 - cos_theta) * v

def project_2D(points, cameraArray):
    """ Convert 3-D points to 2-D by projecting onto images """
    points_proj = rotate(points, cameraArray[:, :3])
    points_proj += cameraArray[:, 3:6]
    points_proj = points_proj[:, :2] / points_proj[:, 2, np.newaxis]
    # points_proj -= cameraArray[:, 9:] / 1778
    f = cameraArray[:, 6]
    k1 = cameraArray[:, 7]
    k2 = cameraArray[:, 8]
    n = np.sum(points_proj ** 2, axis=1)
    r = 1 + k1 * n + k2 * n ** 2
    points_proj *= (r * f)[:, np.newaxis]
    points_proj += cameraArray[:, 9:]
    
    return points_proj

In [ ]:
def project_3D(points, cam_array):